In [1]:
import os
import sys
import comtypes.client

In [2]:
# False,表示脚本将启动一个新的sap2000实例，而不是连接已有的。-- False方案好像已失效，以后自动手动打开软件后，用True方案。
# AttachToInstance = False
AttachToInstance = True
# True则可以手动指定sap2000的安装路径，False则不会执行安装路径的代码，而会自动打开最新安装的sap2000版本。
SpecifyPath = True
ProgramPath = r'C:\Program Files\Computers and Structures\SAP2000 22\SAP2000.exe'
# 设置模型文件的保存路径。
APIPath = r'C:\Users\Administrator\Desktop\SAP'
if not os.path.exists(APIPath): # 如果不存在，则创建
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
# 拼接，os.sep: 操作系统的分隔符（在windows下是 \
ModelPath = APIPath + os.sep + 'API_6-001.sdb'
if AttachToInstance:
    #attach to a running instance of SAP2000
    # 连接到已打开SAP2000实例
    try:
        #get the active SapObject
        # 尝试获取当前正在运行的SAP2000实例的对象，"CSI.SAP2000.API.SapObject"是SAP2000在系统注册表中的COM接口名。
        mySapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    #create API helper object
    # 创建一个新的实例
    # 'SAP2000v22.Helper' SAP2000提供的COM辅助类，用来启动主程序、创建对象等。
    helper = comtypes.client.CreateObject('SAP2000v22.Helper')
    # 调用QueryInterface将对象转换为具体的cHelper接口，这样才能使用其中定义的方法。
    helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
    if SpecifyPath:
        try:
            #'create an instance of the SAPObject from the specified path
            mySapObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try:
            #create an instance of the SAPObject from the latest installed SAP2000
            mySapObject = helper.CreateObjectProgID("CSI.SAP2000.API.SapObject")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
        #start SAP2000 application
        mySapObject.ApplicationStart()



In [19]:
#create SapModel object
# 从SAP2000主对象中提取出SapModel，这是所有结构建模操作的核心对象，之后定义节点、杆件、荷载等操作都通过它来进行。
SapModel = mySapObject.SapModel

In [29]:
#initialize model
SapModel.InitializeNewModel()

0

In [30]:
#create new blank model
ret = SapModel.File.NewBlank()

In [31]:
#switch to k-ft units
kip_ft_F = 4
ret = SapModel.SetPresentUnits(kip_ft_F)

In [32]:
#add frame object by coordinates
LinkName1 = ' '
# 创建link，False是连接两单，否则单点
[LinkName1, ret] = SapModel.LinkObj.AddByCoord(0, 0, 0, 0, 0, 10, LinkName1,False, 'R1', '1', 'Global')

In [35]:
ret = SapModel.View.RefreshView

In [34]:
#assign point object restraint at base
PointName1 = ' '
PointName2 = ' '
Restraint = [True, True, True, True, True,True]
[PointName1, PointName2, ret] = SapModel.LinkObj.GetPoints(LinkName1, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)
ret

[(True, True, True, True, True, True), 0]

In [51]:
## 定义线性的Link属性

In [46]:
# DOF 自由方向矩阵、Fixed 固定、Ke刚度、Ce阻尼。
DOF = [0]*6
Fixed = [0]*6
Ke = [0]*6
Ce = [0]*6

DOF[0] = True
Ke[0] = 4
ret = SapModel.PropLink.SetLinear("L1", DOF, Fixed, Ke, Ce, 0, 0)
ret

[(True, False, False, False, False, False),
 (False, False, False, False, False, False),
 (4.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 0]

In [50]:
# 线弹簧和面弹簧
ret = SapModel.PropLink.SetSpringData("L1", 1, 1)
# 总重量和总质量以及转动惯量
ret = SapModel.PropLink.SetWeightAndMass("L1",0, 1E-5, 0, 0,0)
ret

0

In [53]:
# 绑定link属性到link单元
ret = SapModel.LinkObj.SetProperty(LinkName1, "L1")


In [57]:
# 给节点添加质量
mass = [0]*6
mass[2] = 1 # u3方向加质量
ret = SapModel.PointObj.SetMass(PointName2,mass) # True 覆盖 False 叠加
ret

[(0.0, 0.0, 1.0, 0.0, 0.0, 0.0), 0]

In [60]:
# 添加一个工况模式，
ret = SapModel.LoadPatterns.Add("P",8)


In [63]:
# 给节点添加荷载
value = [0]*6
value[2]=8
ret = SapModel.PointObj.SetLoadForce(PointName2, "P", value)
ret

[(0.0, 0.0, 8.0, 0.0, 0.0, 0.0), 0]

In [64]:
# 定义时程函数
ret = SapModel.Func.FuncTH.SetRamp("TH-1", 3.1416, 1, 20)
ret = SapModel.Func.FuncTH.SetRamp("TH-2", 1.5708, 1, 20)
ret


0

In [65]:
# 模态工况
ret = SapModel.LoadCases.ModalEigen.SetCase("MODAL")
ret


0

In [90]:
# 设置线性的模态时程分析
ret = SapModel.LoadCases.ModHistLinear.SetCase("MHISTA")
    #设置加载的是哪个模式
ret = SapModel.LoadCases.ModHistLinear.SetModalCase("MHISTA", "MODAL")

# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-1']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.ModHistLinear.SetLoads("MHISTA", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.ModHistLinear.SetTimeStep("MHISTA", 100, 0.1)
# 阻尼
ret = SapModel.LoadCases.ModHistLinear.SetDampConstant("MHISTA", 0)


In [91]:
# 设置线性的模态时程分析
ret = SapModel.LoadCases.ModHistLinear.SetCase("MHISTB")
    #设置加载的是哪个模式
ret = SapModel.LoadCases.ModHistLinear.SetModalCase("MHISTB", "MODAL")

# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-2']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.ModHistLinear.SetLoads("MHISTB", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.ModHistLinear.SetTimeStep("MHISTB", 100, 0.1)
# 阻尼
ret = SapModel.LoadCases.ModHistLinear.SetDampConstant("MHISTB", 0)

In [92]:
# 设置线性的模态时程分析
ret = SapModel.LoadCases.ModHistLinear.SetCase("MHISTC")
    #设置加载的是哪个模式
ret = SapModel.LoadCases.ModHistLinear.SetModalCase("MHISTC", "MODAL")

# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-2']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.ModHistLinear.SetLoads("MHISTC", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.ModHistLinear.SetTimeStep("MHISTC", 100, 0.1)
# 阻尼
ret = SapModel.LoadCases.ModHistLinear.SetDampConstant("MHISTC", 0.999)

In [104]:
# 线性直接积分时程分析
ret = SapModel.LoadCases.DirHistLinear.SetCase("DHISTA1")
# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-1']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.DirHistLinear.SetLoads("DHISTA1", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.DirHistLinear.SetTimeStep("DHISTA1", 40, 0.25)
# 阻尼


In [103]:
# 线性直接积分时程分析
ret = SapModel.LoadCases.DirHistLinear.SetCase("DHISTA2")
# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-1']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.DirHistLinear.SetLoads("DHISTA2", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.DirHistLinear.SetTimeStep("DHISTA2", 4000, 2.5E-3)
# 阻尼


In [102]:
# 线性直接积分时程分析
ret = SapModel.LoadCases.DirHistLinear.SetCase("DHISTB1")
# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-2']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.DirHistLinear.SetLoads("DHISTB1", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.DirHistLinear.SetTimeStep("DHISTB1", 40, 0.25)
# 阻尼

In [101]:
# 线性直接积分时程分析
ret = SapModel.LoadCases.DirHistLinear.SetCase("DHISTB2")
# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-2']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.DirHistLinear.SetLoads("DHISTB2", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.DirHistLinear.SetTimeStep("DHISTB2", 4000, 2.5E-3)
# 阻尼


In [100]:
# 线性直接积分时程分析
ret = SapModel.LoadCases.DirHistLinear.SetCase("DHISTC1")
# 定义数组参数
# 施加荷载
numbers_loads = 1 # 一个荷载项
MyLoadType = ['LOAD'] # 荷载类型
MyLoadName = ['P'] # 荷载模式名
MyFunc = ['TH-2']
MySF = [1.0] # 比例系数
MyTF = [1] # 时间系数
MyAT = [0] # 到达时间
# 只有当MyLoadType为Accel时才生效
MyCSys = [' '] # 空字符串，默认全局坐标
MyAng = [0.0] # 加速度方向角

ret = SapModel.LoadCases.DirHistLinear.SetLoads("DHISTC1", 1, MyLoadType, MyLoadName, MyFunc, MySF, MyTF, MyAT, MyCSys, MyAng)
# 时间步
ret = SapModel.LoadCases.DirHistLinear.SetTimeStep("DHISTC1", 40, 0.25)
# 阻尼
ret = SapModel.LoadCases.DirHistLinear.SetDampProportional("DHISTC1", 1,3.0312, 0.2412, 0, 0, 0, 0)


In [98]:
#save model
ret = SapModel.File.Save(ModelPath)